In [ ]:
!pip install tweepy
import tweepy
import tweepy
import time
from textblob import TextBlob


import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

import nltk
nltk.download('all')

!pip install NRCLex
from nrclex import NRCLex

In [ ]:
consumerKey = "6bDyiwANMUAyfThjAPLXMEmbI"
consumerSecret = "cPVwuqxv8JnN4L41IwWgcRYaLhEeGub8mYGzlTJzw1lTDDNeYO"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAFc0ggEAAAAAJUlQ2Itfjttg2%2BsynZ4CWNMVU9s%3DceUCxm1EHpsNws84B77DAMrNpgJfbVVxg6MWOh5VLmRbPbm3Ko"

accessToken = "1564594719904120836-ZSdYGzkk1xZp36hIDHpIGQYaCFcKnj"
accessTokenSecret = "9fzywTS4zI91SASBSn3sHjC43Piu0JB0hqjXCIBsbOlbk"


In [ ]:
#Create the authentication object

auth = tweepy.OAuthHandler(
   consumerKey, consumerSecret
)
auth.set_access_token(accessToken, accessTokenSecret)
auth.secure = True

api = tweepy.API(auth, wait_on_rate_limit = True)


client = tweepy.Client(bearer_token = bearer_token)


In [ ]:
##Tweets count


client = tweepy.Client(bearer_token = bearer_token)

query = 'BTC OR $BTC -is:retweet'

counts = client.get_all_tweets_count(query=query, start_time='2021-11-04T00:00:00Z', end_time='2022-06-20T00:00:00Z', granularity = 'day')

r=0

l=0

for count in counts.data:
    r=r+count['tweet_count']
    l=l+1
    #print(count)

print(r)
print(l)


##Tweets count with Paginator



r=0
l=0
for tweet in tweepy.Paginator(client.get_all_tweets_count, "(Bitcoin) -rt -retweet -winner -giveaway -follow -is:retweet lang:en -has:media -has:images -has:videos -is:reply -has:links", granularity = 'day', start_time='2022-02-01T00:00:00Z', end_time='2022-04-05T00:00:00Z').flatten():
    r=r+tweet['tweet_count']
    l=l+1
    #print(tweet['tweet_count'])
    #print(tweet)


print(r)
print(l)

In [ ]:
##All tweets API Paginator


query = "(Bitcoin) -rt -retweet -winner -giveaway -follow -is:retweet lang:en -has:media -has:images -has:videos -is:reply -has:links"

tweets = tweepy.Paginator(client.search_all_tweets,
                          query=query,
                          user_fields = ['username', 'public_metrics', 'description', 'location'],
                          expansions = 'author_id',
                          start_time='2022-02-01T00:00:00Z', end_time='2022-04-05T00:00:00Z',
                          max_results = 500,
                          tweet_fields=['created_at', 'geo', 'public_metrics', 'text'])
btc_tweets=[]

for response in tweets:
    time.sleep(1)
    btc_tweets.append(response)

print(btc_tweets)


result = []
user_dict = {}
# Loop through each response object
for response in btc_tweets:
    # Taking all of the users, and putting them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username,
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, we will find the author's information
        author_info = user_dict[tweet.author_id]
        # Putting all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id,
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                      })

# Changing this list of dictionaries into a dataframe
df = pd.DataFrame(result)





print(df)





#df to df to csv python

df.to_csv('tweet.csv')
